### [ 손글씨 숫자 인식 모델 구현 ]
- 데이터셋 : mnist_train.csv, mnist_test.csv
- 학습종류 : 지동학습 - 다중클래스분류
- 학습방법 : 인공신경망기반 

[1] 모듈로딩 및 데이터 준비<hr>

In [49]:
# [1-1] 모듈로딩
import sys
sys.path.append('../Utils')
import utils as uf                                      # 함수 가져오기

import torch                                            # 텐서 및 수치과학 함수들 관련 모듈
import torch.nn as nn                                   # 신경망 층 관련 모듈
import torch.nn.functional as F                         # 신경망 함수들(AF, LF, MF) 모듈
import torch.optim as optim                             # 신경망 최적화 모듈


from torch.utils.data import Dataset, DataLoader        # pytorch의 데이터 로딩
from torch.utils.data import Subset                     # pytorch의 데이터셋 관련 모듈
from sklearn.model_selection import train_test_split
import pandas as pd


In [50]:
# [1-2] 데이터 준비
TRAIN_FILE = '../Data/mnist_train.csv'
TEST_FILE  = '../Data/mnist_test.csv'

## [1-3] 데이터 로딩
trainDF = pd.read_csv(TRAIN_FILE, header=None)
testDF  = pd.read_csv(TEST_FILE, header=None)


[2] 커스텀 데이터셋 준비 <hr>

In [51]:
## --------------------------------------------------------------------------------
## [2-1] 커스텀 데이터셋 클래스 정의
## --------------------------------------------------------------------------------
## 클래스이름 : ClfDataset
## 부모클래스 : Dataset
## 오버라이딩 : _ _init_ _(self)        : [필수] 피쳐, 타겟, [선택]행수, 컬럼수, 타겟 수...
##            _ _len_ _(self)          : len() 내장함수 실행 시 자동 호출, 샘플 수 반환
##            _ _getitem_ _(self, idx) : 인스턴스명[idx] 시 자동 호출,
##                                       idx에 해당하는 피쳐, 타겟을 텐서화 해서 반환
## --------------------------------------------------------------------------------
class ClfDataset(Dataset):

    ##- 피쳐와 타겟 저장 및 기타 속성 초기화 
    def __init__(self, dataDF):
        super().__init__()
        ## 피쳐, 타겟 초기화 필수
        self.x = dataDF[dataDF.columns[1:]].values
        self.y = dataDF[dataDF.columns[0]].values


    ##- 데이터 샘플 수 반환 메서드 : len() 함수에 자동호출됨
    def __len__(self):
        return self.x.shape[0] 
    
    ##- 인덱스에 해당하는 피쳐와 타겟 텐서 반환 메서드 : 인스턴스명[index]에 자동호출됨
    def __getitem__(self, index):
        xTS = torch.tensor(self.x[index], dtype=torch.float32)
        xTS = xTS.view(1, 28, 28)
        xTS = xTS / 255.0 ## 정규화
        
        yTS = torch.tensor(self.y[index], dtype=torch.long)
        return xTS, yTS



In [52]:
## --------------------------------------------------------------------------------
## [2-2] 커스텀 데이터셋 인스턴스 생성 및 사용
## --------------------------------------------------------------------------------
allDS   = ClfDataset(trainDF)
testDS  = ClfDataset(testDF)

print(f'allDS : {len(allDS)},  testDS : {len(testDS)}')
print(testDS[0][0].shape)

allDS : 60000,  testDS : 10000
torch.Size([1, 28, 28])


In [53]:
a2 = testDS[0][0].reshape(-1, 28)
a2.unsqueeze_(0).shape
a2.shape

torch.Size([1, 28, 28])

In [54]:
## [2-3] 학습용/검증용/테스트용 데이터셋 분리

## 학습용 데이터셋에서 타겟/라벨만 추출 
targetList = allDS.y 
dataIndexList =list( range(len(allDS)))

## 학습용/검증용 데이터셋 인덱스 분리
## -train_test_split() 함수 : train:test = 75:25 비율로 학습용, 테스트용 데이터셋 분리
##                            stratify : 분류용 데이터셋 경우 카테고리 비율 유지해서
##                                       데이터셋 분리
X_trainIdx, X_validIdx , y_train, y_valid = train_test_split( dataIndexList,
                                                              targetList,
                                                              train_size=0.8,
                                                              stratify=targetList,
                                                              random_state=10 )


In [55]:
## -----------------------------------------------------------------
## 학습용/검증용 데이터셋 생성 ===> Dataset ==> 2개 Subset 분리
## -----------------------------------------------------------------
trainDS = Subset(allDS, X_trainIdx)
validDS = Subset(allDS, X_validIdx)

print(f'allDS   : {type(allDS)},   {len(allDS)}개')
print(f'trainDS : {type(trainDS)}, {len(trainDS)}개')
print(f'validDS : {type(validDS)}, {len(validDS)}개')

allDS   : <class '__main__.ClfDataset'>,   60000개
trainDS : <class 'torch.utils.data.dataset.Subset'>, 48000개
validDS : <class 'torch.utils.data.dataset.Subset'>, 12000개


[3] 모델 클래스 만들기 <hr>

In [56]:
## --------------------------------------------------------------------
## [3-1] 커스텀 모델 클래스 정의
## --------------------------------------------------------------------
##                 입력             커널      패딩    스트라이드         출력            활성화 함수
## 입력층      (BS,1,28,28)          -         -         -         (BS,1,28,28)           -
## Conv       (BS,1,28,28)        (3x3)10     -       (1,1)       (BS,10,26,26)         relu
## Pool       (BS,1,28,28)        (2x2)1      -       (2,2)       (BS,10,13,13)          -
## flatten    (BS,10,13,13)         -         -         -         (BS,10x13x13)          -
## 출력층      (BS,10x13x13)       퍼셉트론10   -         -          (BS,10)            lossFN따라
## --------------------------------------------------------------------
## 클래스이름 : MNISTModel
## 부모클래스 : nn.Module
## 오버라이딩 : __init__(self)
##            forward(self, x)
## 
## --------------------------------------------------------------------
class MNISTModel(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv_layer = nn.Conv2d(1, 10, 3)    ## 흑백, 커널 10개. 커널 3*3, 스트라이드 1, 패딩 x
        self.pool_layer = nn.MaxPool2d(2, 2)     ## Downsampling 크기 즉, 행과 열 1/2 줄임
        self.flatten_layer = nn.Flatten()        ## 4D(BS, C, H, W) -> 2D (BS, C*H*W)
        self.out_layer = nn.Linear(1690, 10)     ## (C*H*W, 10) 
    
    ## 순방향 학습 진행 함수
    def forward(self, x) :
        # - data 입력층 -> 은닉층 1 : conv2d
        out = F.relu(self.conv_layer(x))
        
        # - 은닉층 2 : MaxPooling2D
        out = self.pool_layer(out)
        
        #- 은닉층 3 : Flatten
        out = self.flatten_layer(out)         
        
        #- 출력층 
        out = self.out_layer(out)
        return out    
        

In [57]:
## --------------------------------------------------------------------
## [3-2] 커스텀 모델 클래스 구조 확인 -> torchinfo 활용
## --------------------------------------------------------------------
from torchinfo import summary

model = MNISTModel()
summary(model, input_size = (1, 1, 28, 28))

Layer (type:depth-idx)                   Output Shape              Param #
MNISTModel                               [1, 10]                   --
├─Conv2d: 1-1                            [1, 10, 26, 26]           100
├─MaxPool2d: 1-2                         [1, 10, 13, 13]           --
├─Flatten: 1-3                           [1, 1690]                 --
├─Linear: 1-4                            [1, 10]                   16,910
Total params: 17,010
Trainable params: 17,010
Non-trainable params: 0
Total mult-adds (M): 0.08
Input size (MB): 0.00
Forward/backward pass size (MB): 0.05
Params size (MB): 0.07
Estimated Total Size (MB): 0.13

In [58]:
MNISTModel()

MNISTModel(
  (conv_layer): Conv2d(1, 10, kernel_size=(3, 3), stride=(1, 1))
  (pool_layer): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (flatten_layer): Flatten(start_dim=1, end_dim=-1)
  (out_layer): Linear(in_features=1690, out_features=10, bias=True)
)

[4] 학습 준비 <hr>

In [59]:
## ------------------------------------------------------------
## [4-1] 학습 관련 설정
## ------------------------------------------------------------
## 학습 진행 횟수 및 학습량, 학습 진행 위치, W/b 업데이트 간격
EPOCHS      = 100
BATCH_SIZE  = 200
LR          = 0.01
DEVICE      = 'cuda' if torch.cuda.is_available() else "cpu"


In [60]:
## ------------------------------------------------------------
## [4-2] 학습 관련 인스턴스 생성
## ------------------------------------------------------------
## -> Model 인스턴스 : 자동으로 층별 W, b 텐서 생성 및 랜덤 초기화
model = MNISTModel().to(DEVICE)

## -> 손실함수 인스턴스 : 다중분류용
lossFn = nn.CrossEntropyLoss()

## -> 최적화 인스턴스 : 모델의 층별 파라미터 즉, W, b 업데이트
optim = optim.Adam(model.parameters(), lr= LR)

## -> 데이터로더 인스턴스 : 학습/검증/테스트에 사용될 학습량 만큼 데이터 추출
trainDL = DataLoader(trainDS, batch_size=BATCH_SIZE, shuffle=True)
validDL = DataLoader(validDS, batch_size=BATCH_SIZE)
testDL = DataLoader(testDS, batch_size=BATCH_SIZE)


[5] 학습 진행 <hr>

In [61]:
import copy

patience = 10
min_delta = 1e-4

best_valid_loss = float("inf")
wait = 0
best_state_dict = None
best_epoch = -1


In [ ]:
### --> 학습 진행
## - 학습과 검증 결과 저장 : 학습 진행/중단 여부 결정, 모델 저장 여부
TRAIN_LA = {'loss' : [] , 'acc' : []}
VALID_LA = {'loss' : [] , 'acc' : []}

## 지정된 학습 횟수 만큼 학습 진행 & 에포크 단위로 학습과 검증 결과 저장
for epoch in range(EPOCHS):
    ## 1 에포크 학습
    train_loss, train_acc = uf.train_one_epoch(model, trainDL, lossFn, optim, DEVICE)
    
    ## 1에포크 학습 후 업데이트 W, b 검사 : 학습에 사용되지 않는 데이터
    valid_loss, valid_acc = uf.evaluate(model, validDL, lossFn, DEVICE)
    
    ## 학습과 검증 결과 저장
    TRAIN_LA['loss'].append(train_loss)
    TRAIN_LA['acc'].append(train_acc)
    VALID_LA['loss'].append(valid_loss)
    VALID_LA['acc'].append(valid_acc)
    
    ## 진행 상황 출력
    print(f'[EPOCH - {epoch:03}] TRAIN => Loss : {train_loss:.7f}   Acc : {train_acc:.5f}')
    print(f'{" "*14}VALID=> Loss : {valid_loss:.7f}   Acc : {valid_acc:.5f}')
    
     # 3) Early Stopping (if문)
    # 개선 조건: valid_loss가 best보다 min_delta 이상 감소
    if valid_loss < best_valid_loss - min_delta:
        best_valid_loss = valid_loss
        wait = 0
        best_epoch = epoch

        # best 모델 저장(메모리)
        best_state_dict = copy.deepcopy(model.state_dict())

    else:
        wait += 1
        print(f'증가 없음 {wait}/{patience}')

        if wait >= patience:
            print(f'멈춤! best_epoch={best_epoch}, best_valid_loss={best_valid_loss:.6f}')
            break

    

[EPOCH - 000] TRAIN => Loss : 0.3371094   Acc : 0.90129
              VALID=> Loss : 0.1474394   Acc : 0.95483
[EPOCH - 001] TRAIN => Loss : 0.1101117   Acc : 0.96854
              VALID=> Loss : 0.0914086   Acc : 0.97208
[EPOCH - 002] TRAIN => Loss : 0.0820413   Acc : 0.97535
              VALID=> Loss : 0.0863746   Acc : 0.97267
[EPOCH - 003] TRAIN => Loss : 0.0679215   Acc : 0.97960
              VALID=> Loss : 0.0764853   Acc : 0.97675
[EPOCH - 004] TRAIN => Loss : 0.0563177   Acc : 0.98227
              VALID=> Loss : 0.0757359   Acc : 0.97625
[EPOCH - 005] TRAIN => Loss : 0.0542643   Acc : 0.98315
              VALID=> Loss : 0.0723843   Acc : 0.97750
[EPOCH - 006] TRAIN => Loss : 0.0467668   Acc : 0.98460
              VALID=> Loss : 0.0702444   Acc : 0.97917
[EPOCH - 007] TRAIN => Loss : 0.0410356   Acc : 0.98650
              VALID=> Loss : 0.0738995   Acc : 0.97900
⏳ No improvement. wait=1/10
[EPOCH - 008] TRAIN => Loss : 0.0401446   Acc : 0.98752
              VALID=> Loss :